# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: No module named 'pymorphy2'

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import nltk

# Прочитаем данные из файла с правильной кодировкой
with open('litw-win.txt', 'r', encoding='windows-1251') as f:
    lines = f.readlines()

# Объединим строки в один текст
text = ' '.join(line.split()[1] for line in lines if len(line.split()) > 1)

# Токенизируем текст
words = nltk.word_tokenize(text)

# Получим уникальные слова  
unique_words = set(words)

# Выведем количество уникальных слов и сами слова
unique_words


{'раскосившиеся',
 'препя',
 'окруженному',
 'векторе',
 'поблагословить',
 'обрадовала',
 'чашечки',
 'пугнет',
 'подошл',
 'разломил',
 'шуметь',
 'маже',
 'франц',
 'проскандировал',
 'цветки',
 'каторги',
 'тростника',
 'затаит',
 'стаканчик',
 'запрестольный',
 'прядях',
 'земля',
 'воспели',
 'думае',
 'залаяла',
 'примиряющий',
 'провинциальным',
 'демытрович',
 'порочных',
 'клеветать',
 'неудобно',
 'кипели',
 'сирень',
 'аверкии',
 'свидригайлове',
 'погулять',
 'съел',
 'памятны',
 'кривляются',
 'уклоненной',
 'подносом',
 'познаниям',
 'сук',
 'зальет',
 'студодейное',
 'бисаврюк',
 'сквозила',
 'чертовская',
 'неуловим',
 'косвенных',
 'воо',
 'чудосия',
 'предприимчивому',
 'отскочи',
 'злобою',
 'изловчавшегося',
 'впускай',
 'сокращения',
 'разгинаясь',
 'романтические',
 'степановы',
 'вырастала',
 'лихорадочный',
 'опустошали',
 'потупляли',
 'гложет',
 'смягчилась',
 'всеобщее',
 'псу',
 'воскре',
 'седоватою',
 'одея',
 'огородник',
 'словил',
 'соображаю',
 'помещ

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [2]:
import random
from nltk.metrics import edit_distance

# Преобразуем уникальные слова в список для случайного выбора
unique_words_list = list(unique_words)

# Генерация 5 случайных пар слов
random_pairs = [(random.choice(unique_words_list), random.choice(unique_words_list)) for _ in range(5)]

# Вычисление расстояния редактирования для каждой пары
for word1, word2 in random_pairs:
    distance = edit_distance(word1, word2)
    print(f'Расстояние редактирования между "{word1}" и "{word2}": {distance}')


Расстояние редактирования между "копии" и "сельских": 7
Расстояние редактирования между "построенной" и "улепетывайте": 12
Расстояние редактирования между "библейским" и "вторгнутся": 10
Расстояние редактирования между "когти" и "вознагражден": 10
Расстояние редактирования между "лесной" и "заподозренным": 11


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [3]:
def find_k_nearest_words(word, words, k):
    distances = [(w, edit_distance(word, w)) for w in words]
    # Сортируем по расстоянию и выбираем первые k элементов
    nearest_words = sorted(distances, key=lambda x: x[1])[:k]
    return nearest_words

word = "ратники"
k = 3
nearest_words = find_k_nearest_words(word, unique_words, k)
print(nearest_words)

[('ратники', 0), ('латники', 1), ('ратник', 1)]


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [5]:
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# Инициализируем стеммер и лемматизатор
stemmer = SnowballStemmer('russian')
lemmatizer = WordNetLemmatizer()

# Создаем DataFrame
df = pd.DataFrame(words, columns=['word'])

# Применяем стемминг и лемматизацию
df['stemmed_word'] = df['word'].apply(stemmer.stem)
df['normalized_word'] = df['word'].apply(lemmatizer.lemmatize)

# Указываем 'word' в качестве индекса
df.set_index('word', inplace=True)

# Выводим DataFrame
df


,stemmed_word,normalized_word
word,,
и,и,и
в,в,в
я,я,я
с,с,с
а,а,а
...,...,...
высокопревосходительства,высокопревосходительств,высокопревосходительства
попреблагорассмотрительст,попреблагорассмотрительст,попреблагорассмотрительст
попреблагорассмотрительствующемуся,попреблагорассмотрительств,попреблагорассмотрительствующемуся


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [6]:
from nltk.corpus import stopwords
from collections import Counter

# Получаем стоп-слова для русского языка
stop_words = set(stopwords.words('russian'))

# Удаляем стоп-слова
filtered_words = [word for word in words if word.lower() not in stop_words]

# Подсчитываем частоту слов до и после удаления стоп-слов
original_word_counts = Counter(words)
filtered_word_counts = Counter(filtered_words)

# Рассчитываем долю стоп-слов
stop_words_count = len(words) - len(filtered_words)
stop_words_ratio = stop_words_count / len(words)

# Сравниваем топ-10 самых частых слов до и после удаления стоп-слов
top_10_original = original_word_counts.most_common(10)
top_10_filtered = filtered_word_counts.most_common(10)

print(f'Доля стоп-слов: {stop_words_ratio:.2%}')
print('Топ-10 слов до удаления стоп-слов:', top_10_original)
print('Топ-10 слов после удаления стоп-слов:', top_10_filtered)


Доля стоп-слов: 0.09%
Топ-10 слов до удаления стоп-слов: [('и', 1), ('в', 1), ('я', 1), ('с', 1), ('а', 1), ('к', 1), ('у', 1), ('о', 1), ('н', 1), ('п', 1)]
Топ-10 слов после удаления стоп-слов: [('н', 1), ('п', 1), ('б', 1), ('т', 1), ('д', 1), ('м', 1), ('ч', 1), ('з', 1), ('г', 1), ('е', 1)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('recipes_sample.csv')

# Убедимся, что описания рецептов находятся в колонке 'description'
descriptions = df['description'].dropna().tolist()

# Выбираем случайным образом 5 рецептов
random_indices = random.sample(range(len(descriptions)), 5)
random_descriptions = [descriptions[i] for i in random_indices]
recipe_names = [df.loc[i, 'name'] for i in random_indices]  # Получим названия рецептов

# Инициализируем TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразуем тексты в числовые векторы
tfidf_matrix = vectorizer.fit_transform(random_descriptions)

# Преобразуем результат в DataFrame для удобства
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=recipe_names)

# Выведем результат
tfidf_df

,20,about,actually,addict,and,anna,anything,beef,biscuits,bit,...,unhealthy,up,use,was,we,when,will,with,woman,worcestershire
chocolate chip cookie delight,0.000000,0.000000,0.000000,0.304305,0.203796,0.000000,0.000000,0.000000,0.000000,0.245511,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
apricot pumpkin bread pudding diabetic heart healthy,0.130656,0.130656,0.000000,0.000000,0.175004,0.130656,0.000000,0.130656,0.000000,0.000000,...,0.000000,0.000000,0.391968,0.000000,0.130656,0.000000,0.000000,0.000000,0.00000,0.130656
cherry walnut breakfast couscous,0.000000,0.000000,0.112914,0.000000,0.302480,0.000000,0.112914,0.000000,0.112914,0.091099,...,0.112914,0.112914,0.000000,0.225829,0.000000,0.112914,0.112914,0.112914,0.00000,0.000000
corn souffle stouffer s copycat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.63907,0.000000
basic pasta dough no egg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [8]:
from scipy.spatial.distance import cosine

df = pd.read_csv('recipes_sample.csv')

# Убедимся, что описания рецептов находятся в колонке 'description'
descriptions = df['description'].dropna().tolist()
recipe_names = df['name'].dropna().tolist()  # Получим список названий рецептов

# Выбираем случайным образом 5 рецептов
random_indices = random.sample(range(len(descriptions)), 5)
random_descriptions = [descriptions[i] for i in random_indices]
random_recipe_names = [recipe_names[i] for i in random_indices]  # Получим случайные названия рецептов

# Инициализируем TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразуем тексты в числовые векторы
tfidf_matrix = vectorizer.fit_transform(random_descriptions)

# Преобразуем результат в DataFrame для удобства
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=random_recipe_names)

# Вычисляем косинусное расстояние между каждой парой рецептов
distances = pd.DataFrame(index=random_recipe_names, columns=random_recipe_names)

for i in range(len(random_recipe_names)):
    for j in range(len(random_recipe_names)):
        if i == j:
            distances.iloc[i, j] = 0.0
        else:
            distances.iloc[i, j] = cosine(tfidf_matrix.toarray()[i], tfidf_matrix.toarray()[j])

# Преобразуем тип данных в float
distances = distances.astype(float)

# Выведем результат
distances


,campfire or oven pepper jack and bacon potatoes,jamie deen s chicken salad,rainbow risotto,whipped topping dollops on spoons,shrimp marinara
campfire or oven pepper jack and bacon potatoes,0.000000,0.807058,0.914321,0.827754,0.948036
jamie deen s chicken salad,0.807058,0.000000,0.966846,0.922309,0.892112
rainbow risotto,0.914321,0.966846,0.000000,0.935732,0.967262
whipped topping dollops on spoons,0.827754,0.922309,0.935732,0.000000,0.905227
shrimp marinara,0.948036,0.892112,0.967262,0.905227,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Чтобы определить, какие рецепты являются наиболее похожими, нужно проанализировать значения косинусного расстояния в таблице. Чем ближе значение косинусного расстояния к 0, тем более похожи рецепты